In [1]:
%matplotlib inline
import sys
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arff

from sklearn import tree
from sklearn import linear_model
from sklearn.externals.six import StringIO  
from IPython.core.display import Image

import pydot

# Load the Data

In [2]:
%%time
gender = {'Male' : 0, 'Female' : 1, 'Unknown' : 2}
language = {'English' : 0, 'EnglishandAnother' : 1, 'Another' : 2}

trainfile = 'sample_data\\test05.arff'

examples = []
features = []
labels = []

with open(trainfile, 'rb') as af:
    arffFile = arff.load(af)
    # do not include college gpa and college credits
    features = [arffFile['attributes'][0:-3][i][0].encode("ascii") for i in range(len(arffFile['attributes'][0:-3]))]
    data = arffFile['data']
    for row in data:
        row[0] = gender[row[0]] # index of gender
        row[12] = language[row[12]] # index of language
        examples.append(row[0:-3]) # do not include number of credits taken first year
        labels.append(row[-1])
        
print 'Number of examples:', len(examples)

Number of examples: 2801
Wall time: 236 ms


# Perceptron

In [3]:
%%time
clf = linear_model.Perceptron()
print clf.fit(examples, labels)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=5, n_jobs=1, penalty=None, random_state=0, shuffle=True,
      verbose=0, warm_start=False)
Wall time: 16 ms


In [4]:
print clf.coef_

[[-2.          1.          0.59149208  0.66892755 -3.8834674  -0.69555671
  -1.67831756 -2.0598203   0.94188256 -0.96236631 -2.28516058 -2.18143143
  -1.          0.79972382]
 [-1.         -1.         -0.84854431 -3.82453585  0.39345071 -3.72635443
  -2.67148022  0.72998589  2.89144349  0.24407558 -3.83664701 -0.98235822
  -1.          0.55076099]
 [ 0.          1.          0.47567226 -1.1494686  -0.13057386 -1.48918923
  -1.17355821 -3.20579777 -3.72546542  0.18720128 -3.07584502 -0.37309302
   0.         -1.50881369]
 [-1.          1.          1.49883655  1.4285634  -2.42973493 -5.46088886
  -2.75812625 -2.79166706 -2.20230909  0.56919426 -2.00299878 -1.66574425
  -2.         -0.06416229]
 [-2.         -1.          2.5394573  -0.23269745 -2.04647468 -2.80067367
  -2.0711604   2.83995262 -4.60705236  5.09562028  0.56290539  1.80996897
   0.         -5.35754691]
 [ 1.          1.         -2.42070473 -0.67924893 -1.4919875  -0.84642045
  -1.44916311 -2.26017263 -0.33064796 -2.29327593 -

# Decision Tree

In [5]:
%%time
dt = tree.DecisionTreeClassifier()
dt = dt.fit(examples, labels)
print dt

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')
Wall time: 28 ms


In [6]:
%%time
dot_data = StringIO()
tree.export_graphviz(dt, out_file=dot_data, feature_names=features) 
#tree.export_graphviz(dt, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
treepng = graph.create_png()

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.909563 to fit

Wall time: 27 s


In [7]:
Image(treepng)

In [8]:
%%time
with open('picture_out.png', 'wb') as f:
    f.write(treepng)

Wall time: 109 ms


# Evaluate Tree on Test Set

In [9]:
%%time
trainfile = 'sample_data\\train05.arff'

testexamples = []
testlabels = []

with open(trainfile, 'rb') as af:
    arffFile = arff.load(af)
    # do not include college gpa and college credits
    features = [arffFile['attributes'][0:-3][i][0].encode("ascii") for i in range(len(arffFile['attributes'][0:-3]))]
    data = arffFile['data']
    for row in data:
        row[0] = gender[row[0]] # index of gender
        row[12] = language[row[12]] # index of language
        testexamples.append(row[0:-3]) # do not include number of credits taken first year
        testlabels.append(row[-1])

Wall time: 184 ms


In [10]:
%%time
numCorrect = 0
numIncorrect = 0
testCount = len(testexamples)
for index in range(testCount):
    x = testexamples[index]
    if dt.predict(x) == testlabels[index]:
        numCorrect += 1
    else:
        numIncorrect += 1
print 'Correctly Classified:', numCorrect
print 'Incorrectly Classified:', numIncorrect
print 'Error: %f' % (float(numCorrect) / float(testCount) * 100)

Correctly Classified: 57
Incorrectly Classified: 25
Error: 69.512195
Wall time: 4 ms
